# Tutorial on Einstein's Summation Convention by using Numpy

In [2]:
import matplotlib.pyplot as plt
import imageio
import os
import sys
import skimage as ski
import time
import numpy as np

print(os.getcwd())
sys.path.append(os.path.abspath("../src/"))
from inverse_compositional_algorithm import inverse_compositional_algorithm, robust_inverse_compositional_algorithm, pyramidal_inverse_compositional_algorithm

/mnt/git/computational_astro/inverse_compositional_algorithm/test


## Some examples of Einstein's Summation Convention

### 1. Matrix transpose

In [3]:
a = np.arange(6).reshape(2,3)

print("a = ")
print(a)

b = np.einsum("ij->ji", a)
print("b = a.T")
print(b)

a = 
[[0 1 2]
 [3 4 5]]
b = a.T
[[0 3]
 [1 4]
 [2 5]]


### 2. Sum

In [4]:
# Sum of all matrix elements
c = np.einsum("ij->", a)
print("c = sum(a)")
print(c)

c = sum(a)
15


In [5]:
# Column sum
d = np.einsum("ij->j", a)
print("d = sum(a, axis=0)")
print(d)

# Row sum
e = np.einsum("ij->i", a)
print("e = sum(a, axis=1)")
print(e)

d = sum(a, axis=0)
[3 5 7]
e = sum(a, axis=1)
[ 3 12]


### 3. Matrix vector multiplication

In [6]:
b = np.arange(3)
print("b = ")
c = np.einsum("ij,j->i", a, b)
print("c = a*b")
print(c)

b = 
c = a*b
[ 5 14]


### 4. Matrix matrix multiplication

In [7]:
b = np.arange(15).reshape(3,5)
print("a = ")
print(a)
print("b = ")
print(b)
c = np.einsum("ik,kj->ik", a, b)
print("c = a*b")
print(c)

a = 
[[0 1 2]
 [3 4 5]]
b = 
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
c = a*b
[[  0  35 120]
 [ 30 140 300]]


### 5. Dot product

In [8]:
# For vectors
a = np.arange(3)
b = np.arange(3, 6)
print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("i,i->", a, b)
print("c = sum(a*b)")
print(c)

a = 
[0 1 2]
b = 
[3 4 5]
c = sum(a*b)
14


In [9]:
# For matrices
a = np.arange(6).reshape(2,3)
b = np.arange(6, 12).reshape(2,3)

print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("ij,ij->", a, b)
print("c = sum(a*b)")
print(c)

a = 
[[0 1 2]
 [3 4 5]]
b = 
[[ 6  7  8]
 [ 9 10 11]]
c = sum(a*b)
145


### 6. Hadamard product

In [11]:
a = np.arange(6).reshape(2, 3)
b = np.arange(6, 12).reshape(2, 3)

print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("ij,ij->ij", a, b)
print("Hadamard product c = a*b")
print(c)

a = 
[[0 1 2]
 [3 4 5]]
b = 
[[ 6  7  8]
 [ 9 10 11]]
Hadamard product c = a*b
[[ 0  7 16]
 [27 40 55]]


### 7. Outer product

In [12]:
a = np.arange(3)
b = np.arange(3, 7)

c = np.einsum("i,j->ij", a, b)
print("Outer product, c = a*b")
print(c)

Outer product, c = a*b
[[ 0  0  0  0]
 [ 3  4  5  6]
 [ 6  8 10 12]]


### 8. Batch matrix multiplication

In [13]:
a = np.random.rand(3, 2, 5)
b = np.random.rand(3, 5, 3)

print("a = ")
print(a)
print("b = ")
print(b)

c = np.einsum("Bjk,Bkl->Bjl", a, b)
print("Batch product, c = a*b")
print(c)

a = 
[[[0.26574499 0.96375379 0.10664884 0.8827938  0.35155099]
  [0.35167092 0.5141475  0.59166279 0.53327699 0.80162638]]

 [[0.210634   0.05336498 0.98764853 0.21863412 0.38206155]
  [0.33739639 0.84499918 0.71697605 0.26458939 0.6067253 ]]

 [[0.58678884 0.83526179 0.30621646 0.91471405 0.33318097]
  [0.59809861 0.49098382 0.16880889 0.09340197 0.84946205]]]
b = 
[[[0.04980903 0.38281431 0.6245837 ]
  [0.55202353 0.79026117 0.65861854]
  [0.299957   0.66668573 0.10121439]
  [0.35535389 0.2300061  0.67657496]
  [0.26594458 0.17245631 0.51135077]]

 [[0.61695157 0.42493958 0.04592028]
  [0.91725883 0.62831256 0.30647263]
  [0.79185723 0.38656478 0.55154587]
  [0.44946022 0.79869634 0.655368  ]
  [0.26201167 0.16140316 0.23142675]]

 [[0.17335215 0.99193909 0.24577536]
  [0.42920305 0.71579762 0.49258802]
  [0.17621623 0.8786265  0.27967139]
  [0.85459688 0.71720114 0.91735243]
  [0.25464325 0.27366403 0.12876578]]]
Batch product, c = a*b
[[[0.98443862 1.19812459 1.58856255]
  [0.8815

### 9. A more complex example

We will compute the independant vector equal to the sum over x, of the steepest descent image DIJ with the difference image DI, where x is a position in the image.

For the sake of simplicity of this example, we assume that images have only one (color) channel, and that the parameters registered in DIJ are those of a translation, ie. only two parameters per pixel.

In [19]:
dij = np.arange(72).reshape(4, 3, 3, 2)
di = np.arange(36).reshape(4, 3, 3)
dijt = np.einsum("ijlk->ijkl", dij)
print("dijt shape = ", dijt.shape)
prod = np.einsum("ijkl, ijl -> ijk", dijt, di)
print("prod shape = ", prod.shape)
b = np.einsum("ijl -> l", prod)
print("b shape = ", b.shape)
print(b)

dijt shape =  (4, 3, 2, 3)
prod shape =  (4, 3, 2)
b shape =  (2,)
[29820 30450]
